<a href="https://colab.research.google.com/github/envgp/taking_the_pulse_ocean/blob/master/pulse_ocean_with_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Taking the Pulse of the Oceans & Fishing Activity

2023-01-26

Shin Nakayama (shinn1@stanford.edu) & Fio Micheli (micheli@stanford.edu)

[Stanford Center for Ocean Solutions](https://oceansolutions.stanford.edu), Doerr School of Sustainability

This lab is supposed be submitted by January 31, 2023. You are supposed to put your answer questions within this notebook and share the completed notebook with shinn1@stanford.edu using the `Share` banner located at the top right corner of this notebook. When sharing your notebook, please change the name of the notebook and add your name and sunetid (e.g., `3_hydrosphere_oceans_firstname_lastname_sunetid.ipynb`)

## INTRODUCTION TO THE ASSIGNMENT
Fisheries play a vital role in our food security, economy and ecosystem, while posing significant impacts on the oceans. Industrialization of fisheries has allowed us to catch more fish with engine-powered fishing fleets and gears. Non-selective fishing gears have let bycatch of unwanted and endangered species, while bottom trawlers have killed coral reefs by dragging the nets over seabeds. Currently, 35% of the fish population is overfished, and a study predicts that there will be no fish in the ocean by 2048, calling for better fisheries management.

Data sets used this assignment are: 1) global fishing effort, 2) regional fishing effort, 3) ocean chlorophyll a and sea surface temperature, and 4) shapefile of exclusive economic zones (EEZ).  
1. `fishing_hours.parquet` &mdash; A pre-processed grid data of daily fishing hours from 2016-01-01 to 2021-12-31, sumed within 1 × 1-degree grids. The original data are composed of locations, timestamp, and fishing hours associated with each AIS signal. Data taken from [Global Fishing Watch](https://globalfishingwatch.org).
    - `date`: year-month-day
    - `lat`: latitude
    - `lon`: longitude
    - `gear`: fishing gear type (trawlers, purse seines, etc.)
    - `flag`: vessel flag in ISO 3 code (that is, State which a vessel is registered to)
    - `fishing_hours`: total fishing hours in a 1 x 1-degree grid
2. `fishing_hours_x.parquet` &mdash; A pre-processed grid data of daily fishing hours from 2016-01-01 to 2021-12-31, sumed within 0.1 × 0.1-degree grids, for specific regions (40 × 40 degrees).
    - `fishing_hours_peru.parquet`: Near Peru with longitude [-100, -60] and latitude [-30, 10]
    - `fishing_hours_w_africa.parquet`: Near West Africa with longitude [-40, 0] and latitude [-10, 30]
    - `fishing_hours_falkland.parquet`: Near Falkland Islands with longitude [-70, -30] and latitude [-70, 30]
    - `fishing_hours_pacific.parquet` : in the Pacific with longitude [140, 180] and latitude [-35, 5]    
3. `modis.parquet` &mdash; A pre-processed grid data of monthly mean measurements by MODIS AQUA sensor from 2016-01 to 2021-12, averaged within 1 × 1-degree grids. Data taken from [NASA MODIS Terra satellite imagery](https://developers.google.com/earth-engine/datasets/catalog/NASA_OCEANDATA_MODIS-Terra_L3SMI). 
    - `month`: year-month
    - `sst`: sea surface temperature (°C)
    - `chlor_a`: chlorophyll a concentration (mg/m<sup>3</sup>)
4. `SAU_EEZ` &mdash; shapefile of Exclusive Economic Zones (EEZ). Data taken from [Sea Around US](https://www.seaaroundus.org).

### The Learning Goals for the week:
(where the course learning goals are in plain text, and the focus this week is in italics)

Students will:
- learn about the ways in which climate change and human activity are impacting planet Earth, *with a focus this week on fishing activities in response to climates.*

- become familiar with the wide range of sensors available to study various components of the Earth system. These include sensors on satellites, aircraft, ground-based platforms, and deployed above or beneath the surface on land or water. *This week we will use vessel-borne GPS data on the Automatic Identification System (AIS) for fishing effort and MODIS satellite imagery data for ocean productivity and temperature.* 

- become familiar with the basic physical principles (resolution, sampling, processing workflows, etc.) common to all sensors. *This week we will use data with different sampling frequencies and resolutions.*

- work with various sources of data, learning how to access, analyze, synthesize, and describe the data to quantify trends; think critically and creatively about how to project these trends into the future. *This week we will describe the spatial and temporal patterns of the data.*

- describe the complex interactions between human activity and various components of the Earth system. *This week we will explore the potential drivers of fishing activities by overlaying different datasets on the same plots and with statistics such as linear regression and cross-correlation.*

- become motivated to think about new sensors and new ways of using sensor data to study the planet. This is always the last question in each assignment.

## OVERVIEW OF THE ASSIGNMENT

### I. Understand Global Patterns of Fishing Activity
With climate changes taking place at an alarming rate, it is necessary to understand how fishing activities respond to climates toward effective fisheries management and sustainable fisheries. Here, we will discuss the spatial and temporal patterns of fishing activities at a global scale. For example, where do you find the hotspots? Is there seasonality? Is there a latitudinal gradient? If so, why?


### II. Explore Drivers Underlying Fishing Effort
Considering that humans are the top fish predators equipped with highly-advanced sensors and gears, understanding the drivers of fishing activity is the first step toward effective fisheries management and sustainable ocean use in the future. Here, we will explore the relationship between fishing effort and biological factors. Further, we will observe how social and geopolitical factors may influence fishing activity.


### III. Taking the Pulse in the Future
We conclude with the usual question – what’s next?

## THE ASSIGNMENT

## I. Understand Global Patterns of Fishing Activity

Fishing activity can be inferred from the automatic information system (AIS). Originally to avoid collision at sea, distant-water fishing fleets are required by International Maritime Organization to transmit their positions at sea. From their movement patterns, we can infer whether a vessel is fishing or non-fishing using a machine-learning approach (see [this paper](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0158248) for details). The AIS data are processed and curated by [Global Fishing Watch](https://globalfishingwatch.org), a non-profitable organization with a mission of transparency for sustainable fisheries.

**Q:** Observe the spatial and temporal patterns of global fishing activities and discuss the possible drivers.

In [ ]:
!pip install xarray numpy pandas geopandas cartopy==0.19.0.post1 ipywidgets statsmodels

In [ ]:
!pip uninstall -y shapely

In [ ]:
!pip install shapely --no-binary shapely

In [ ]:
!git clone https://premonition.stanford.edu/sgkang09/taking_the_pulse_ocean_data.git

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
import seaborn as sns
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import itertools
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load data
fish = pd.read_parquet('./taking_the_pulse_ocean_data/fishing_hours.parquet')
fish.head()

In [ ]:
# standardize fishing hours to unit area
# 1 degree is roughly 111 km at the equator and smaller at higher latitudes
fish['area_km2'] = 111 * 111 * np.cos(fish['lat'] * np.pi / 180)
fish['area_km2'] = fish['area_km2'].astype(float) #convert numpy float to pandas float
fish['fishing_hours_km2'] = fish['fishing_hours']/fish['area_km2']

# convert string to datetime object
fish['date'] = pd.to_datetime(fish['date'], format='%Y-%m-%d')

# sum within grids
summary = fish[['lat', 'lon', 'fishing_hours_km2']].groupby(['lat', 'lon']).sum()
summary.head()

In [ ]:
# convert to xarray object for plotting
foo = summary.to_xarray()
foo['lat'] = foo['lat'].astype(int)
foo['lon'] = foo['lon'].astype(int)
# plot on a log scale
plt.figure(figsize=(16, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.axes.add_feature(cf.LAND, color='#000', linewidth=0);
foo['fishing_hours_km2'].plot(
    cmap="magma_r",
    norm=LogNorm(vmin=0.1, vmax=foo['fishing_hours_km2'].max()),
    ax=ax, 
    add_colorbar=False
)

In [ ]:
# plot total fishing hours over time
foo = fish[['date', 'fishing_hours']].groupby('date').sum()
fig = plt.figure(figsize=(16, 6))
ax = plt.subplot(111)
# sns.lineplot(x=foo.index, y=foo['fishing_hours'],legend=False)
foo.plot(ax=ax)
ax.set_xlabel('Year')
ax.set_ylabel('Fishing hours')
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)
ax.grid(True, which='both', alpha=0.2)
fig.autofmt_xdate()

In [ ]:
# plot latitudinal gradient over time
foo = fish.pivot_table(columns='date', index='lat', values='fishing_hours_km2', aggfunc=np.sum)
fig, ax = plt.subplots(1,1, figsize=(16, 6))
times = list(foo.columns)
lat = list(foo.index)
ax.pcolormesh(times, lat, foo.values, cmap='magma_r')
ax.set_xlabel('Year')
ax.set_ylabel('Latitude')
fmt_month = mdates.MonthLocator()
ax.xaxis.set_minor_locator(fmt_month)
ax.grid(True, which='both', alpha=0.2)
fig.autofmt_xdate()

In [ ]:
# # now look at both latitudinal gradients and tempoeral patterns together

# # plot latitudinal gradient over time
# foo = fish.pivot_table(columns='date', index='lat', values='fishing_hours_km2', aggfunc=np.sum)
# foo = foo.loc[::-1] # inverse rows
# foo.reindex(np.arange(-90, 90, 1)) # expand index range

# # find column positions that are Jan 1
# pos = np.where(foo.columns.strftime('%m-%d') == '01-01')[0]

# plt.figure(figsize=(16, 6))
# ax = sns.heatmap(foo, cmap="magma_r", cbar_kws={'label':'Fishing hours / km2'})

# plt.xlabel('Year')
# plt.ylabel('Latitude')
# plt.yticks([0,45,90,135,180], [90,45,0,-45,-90])
# plt.xticks(pos, foo.columns[pos].strftime('%Y'), rotation=0);

**Q:** Observe the spatial patterns of fishing activities by fishing gear type and discuss the possible drivers. Choose fishing gear from {`squid_jigger`, `drifting_longlines`, `trawlers`, `tuna_purse_seines`}.

In [ ]:
# select a gear and sum within grids
gear_name = 'tuna_purse_seines'
summary = fish.loc[fish['gear']==gear_name, ['lat', 'lon', 'fishing_hours_km2']].groupby(['lat', 'lon']).sum()

# convert to xarray object for plotting
foo = summary.to_xarray()
foo['lat'] = foo['lat'].astype(int)
foo['lon'] = foo['lon'].astype(int)

# plot on a log scale
plt.figure(figsize=(16, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
foo['fishing_hours_km2'].plot(cmap="magma_r",
    norm=LogNorm(vmin=foo['fishing_hours_km2'].max()*1e-3, vmax=foo['fishing_hours_km2'].max()),
    ax=ax, zorder=-1, add_colorbar=False)
ax.axes.add_feature(cf.LAND, color='#000', linewidth=0);

## II. Explore Drivers Underlying Fishing Effort
### II.a. Biological drivers
Considering that humans are the top fish predators equipped with highly-advanced sensors, fishing activity should be strongly correlated with the fish distribution. However, we lack global data on fish distribution to support this. Therefore, we will look at a known correlate of fish abundance &mdash; primary production, which can be measured as chlorophyll a, a pigment in phytoplankton used for photosynthesis.

In this course, we will use NASA MODIS satellite imagery, pre-downloaded and pre-processed. MODIS (Moderate Resolution Imaging Spectroradiometer) satellite measures surface reflectance (color and temperature) in 36 spectral bands at spatial resolutions ranging from 250 m to 1 km, depending on the bands. The swath can cover the entire Earth in 1-2 days. We will use MODIS AQUA data.

**Q:** Test whether chlorophyll a explains fishing activity along latitudes using a linear regression. Do the same for each fishing gear type (`trawlers`, `tuna_purse_seines`, `squid_jiggers`, `drifting_longlines`). 

In [ ]:
# read ocean climate data
modis = pd.read_parquet('./taking_the_pulse_ocean_data/modis.parquet')
modis.head()

In [ ]:
# convert month to datetime
modis['year_month'] = pd.to_datetime(modis['month'], format='%Y-%m')
modis.head()

In [ ]:
# plot a monthly mean
group_month = modis.groupby('year_month')
months = list(group_month.groups.keys())
tmp = group_month.get_group(months[0])
summary = tmp[['lat', 'lon', 'chlor_a', 'sst']].groupby(['lat', 'lon']).mean()

# convert to xarray object for plotting
foo = summary.to_xarray()
foo['lat'] = foo['lat'].astype(int)
foo['lon'] = foo['lon'].astype(int)

# plot on a log scale
fig = plt.figure(figsize=(16, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.axes.add_feature(cf.LAND, color='#000', linewidth=0);
foo['sst'].plot(cmap="viridis", ax=ax, add_colorbar=False)

In [ ]:
# latitudinal trend

# mean chlorophyll a within each latitude
foo = modis[['lat', 'chlor_a']].groupby('lat').mean()

# total fishing hours within each latitude
bar = fish[['lat', 'fishing_hours_km2']].groupby('lat').sum()

# merge on index
foo['fishing_hours_km2'] = bar['fishing_hours_km2']
foo.reset_index(inplace=True)

# plot
fig = plt.figure(figsize=(6, 6))

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.plot('chlor_a', 'lat', data=foo, marker='o')
ax1.title.set_text('Chlorophyll a (mg m-3)')

ax2.plot('fishing_hours_km2', 'lat', data=foo, marker='o')
ax2.title.set_text('Fishing hours (km-2)')
for ax in [ax1, ax2]:
  ax.set_ylim(-90, 90)
# plt.show()

In [ ]:
# linear regression
# Is fishing effort explained by chlorophyll a concentration?

bar = foo[~foo.chlor_a.isnull()].copy()
model = sm.OLS(bar['fishing_hours_km2'], bar['chlor_a'])
result = model.fit()
print(result.summary())

In [ ]:
sns.regplot(x='chlor_a', y='fishing_hours_km2', data=foo);

**Q:** Test how monthly fishing hours are correlated to chlorophyll a using cross correlation analysis.

In [ ]:
# time series

# monthly total fishing hours to make same temporal resolutuions as modis data
fish['year'] = fish['date'].dt.year
fish['month'] = fish['date'].dt.month
fish['year_month'] = pd.to_datetime(fish['year']*100 + fish['month'], format='%Y%m')

fish_monthly = fish[['year_month', 'fishing_hours_km2']].groupby('year_month').sum()

# monthly mean chlorophyll a 
foo = modis[['year_month', 'chlor_a']].groupby('year_month').mean()

foo['fishing_hours_km2'] = fish_monthly['fishing_hours_km2']
foo.reset_index(inplace=True)

# plot
fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax1.plot('year_month', 'chlor_a', data=foo, marker='o')
ax1.set_yscale('log')
ax1.title.set_text('Chlorophyll a (mg m-3)')
ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=2))

ax2.plot('year_month', 'fishing_hours_km2', data=foo, marker='o')
ax2.set_yscale('log')
ax2.title.set_text('Fishing hours km-2')
ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_minor_locator(mdates.MonthLocator(interval=2))
# plt.tight_layout()
# plt.show()

In [ ]:
# cross correlation 
x1 = np.log(foo['chlor_a'])
x2 = np.log(foo['fishing_hours_km2'])

ccf_forward = sm.tsa.stattools.ccf(x1, x2, adjusted=False)
ccf_backward = sm.tsa.stattools.ccf(x2, x1, adjusted=False)[::-1]
ccf_output = np.r_[ccf_backward[:-1], ccf_forward]
            
# plot
ccf = pd.DataFrame({'lag':range(-71,72), 'cor':ccf_output})
plt.figure(figsize=(6, 3))
ax = sns.scatterplot(x='lag', y='cor', data=ccf)
ax.stem(ccf['lag'],ccf['cor'], markerfmt=' ')
ax.set_xlim(-7, 7)
plt.show()

**Q:** See how sea surface temperature in the Indian Ocean influences fishing effort.

In [ ]:
# fishing effort in Indian Ocean

fish_io = fish[(fish.lat >= -50) & (fish.lat <= -20) & (fish.lon >=50) & (fish.lon <= 100)].copy()

# aggregate over longitudes
fish_io = fish_io[['year_month', 'lat', 'fishing_hours_km2']].groupby(['year_month', 'lat']).sum()

# latitudinal gradient over time
fish_io.reset_index(inplace=True)
foo = fish_io.pivot(columns='year_month', index='lat', values='fishing_hours_km2')
foo = foo.loc[::-1] # inverse rows
foo.reindex(np.arange(-50, -19, 1)) # expand index range

# same for SST
modis_io = modis[(modis.lat >= -50) & (modis.lat <= -20) & (modis.lon >=50) & (modis.lon <= 100)].copy()

# latitudinal gradient over time
bar = modis_io.pivot_table(columns='year_month', index='lat', values='sst', aggfunc=np.mean)
bar = bar.loc[::-1] # inverse rows

# plot
pos = np.where(foo.columns.strftime('%m-%d') == '01-01')[0]

plt.figure(figsize=(16, 6))
ax = sns.heatmap(foo, cmap="magma_r", cbar_kws={'label':'Fishing hours/km2'})
a = ax.contour(np.arange(0, bar.shape[1]), np.arange(0, bar.shape[0]), bar, colors='black')
ax.clabel(a, inline=True, fontsize=10)
plt.xlabel('Year')
plt.ylabel('Latitude')
plt.yticks([0,10,20,30], [-20,-30,-40,-50])
plt.xticks(pos, foo.columns[pos].strftime('%Y'), rotation=0);

### II.b. Non-biological drivers
Fishing activities are also shaped by other factors, such as holidays, COVID, and fuel prices. Also, fishing regulations should influence the activity. How strongly do they influence fishing activities?

**Q:** Explore how fishing activities are influenced by social / cultural events.

In [ ]:
# activity of non chinese-flagged vessels
other = fish[(fish['flag']!='CHN') | fish['flag'].isnull()].copy()

# sum each day
other_daily = other[['date', 'fishing_hours']].groupby('date').sum()

fig, ax = plt.subplots(1,1, figsize=(16, 6))
# sns.lineplot(other_daily, x='date', y='fishing_hours')
other_daily.plot(ax=ax)
ax.set_xlabel('Year')
ax.set_ylabel('Fishing hours');

In [ ]:
# activity of chinese-flagged vessels
china = fish[fish['flag']=='CHN'].copy()
# sum each day
china_daily = china[['date', 'fishing_hours']].groupby('date').sum()

fig, ax = plt.subplots(1,1, figsize=(16, 6))
china_daily.plot(ax=ax)

ax.set_xlabel('Year')
ax.set_ylabel('Fishing hours')
ax.legend()

In [ ]:

other_daily.plot??

**Q:** Explore how fishing activities are influenced by geospatial regulations. Choose areas from Peru, Folkland Island, Pacific Islands, and West Africa.

In [ ]:
# plot near Peru

data = pd.read_parquet('./taking_the_pulse_ocean_data/fishing_hours_peru.parquet')

data['area_km2'] = 111 * 111 * np.cos(data['lat'] * np.pi / 180)
data['area_km2'] = data['area_km2'].astype(float)
data['fishing_hours_km2'] = data['fishing_hours']/data['area_km2']

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

# change lat lon to integer (we want to sum within each grid but floating point causes problems when grouping)
data['lat'] = (data['lat'].values * 10).astype(int)
data['lon'] = (data['lon'].values * 10).astype(int)

# sum within each grid
summary = data[['lat', 'lon', 'fishing_hours_km2']].groupby(['lat', 'lon']).sum()
summary.head()

In [ ]:
# read EEZ shapefile
eez = gpd.read_file('./taking_the_pulse_ocean_data/SAU_EEZ')

In [ ]:
from shapely.geometry import Polygon
centers = []
for geom in eez.geometry:
    centers.append(geom.centroid)
poly = Polygon(((-100, -30), (-60, -30), (-60, 10), (-100, 10), (-100, -30)))
# poly = Polygon(((-140, -35), (180, -35), (180, 5), (-140, 10), (-140, -35)))
inds = [poly.contains(center) for center in centers]
eez_peru = eez[inds]

In [ ]:
# plot
# takes a while to plot it up?
foo = summary.to_xarray()
foo['lat'] = 0.1*foo['lat']   # back to 0.1 degree intervals
foo['lon'] = 0.1*foo['lon']

# plot on a log scale
plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-100, -60, -30, 10])     # Peru (left, right, bottom, top)
# ax.set_extent([-70, -30, -70, -30])   # Falkland islands
# ax.set_extent([-40, 0, -10, 30])   # West Africa
# ax.set_extent([140, 180, -35, 5])   # Pacific islands
foo['fishing_hours_km2'].plot(
    cmap="magma_r",
    norm=LogNorm(vmin=0.0001, vmax=foo['fishing_hours_km2'].max()),
    ax=ax, zorder=-1, 
    add_colorbar=True
)
eez_peru['geometry'].boundary.plot(ax=ax, color='black', linestyle='dashed', zorder=-1)
ax.add_feature(cf.LAND, linewidth=0)
ax.add_feature(cf.BORDERS)
ax.add_feature(cf.COASTLINE);

## III. Taking the Pulse in the Future
In this assignment, we worked with what we have in terms of sensors and data analysis workflows. If you were given $1 million to develop a new form of sensor or new form of data analysis, what would you do? This can be research, field work, fabrication of sensors, etc. Think about what you would measure, at what spatial and temporal scale and resolution. Over to you to design the next generation of sensors for taking the pulse of our oceans and fishing activity.

---
## Suuplement
How to access original Global Fishing Watch data & NASA MODIS Terra data
#### Global Fishing Watch
- Online web map on their [website](https://globalfishingwatch.org/our-map/)
- R package `gfwr` ([link](https://github.com/GlobalFishingWatch/gfwr)) &mdash; an R API to download fishing data. Note that this is still at an early development stage.

#### NASA MODIS data
- The easiest way is to access the data on [Google Earth Engine](https://developers.google.com/earth-engine/datasets) through a Python API `earthengine-api` ([link](https://github.com/google/earthengine-api))
- You can also find the data on other repositories including [Amazon Web Service](https://aws.amazon.com/opendata/?wwps-cards.sort-by=item.additionalFields.sortDate&wwps-cards.sort-order=desc) or [Copernicus](https://cds.climate.copernicus.eu), or original data [NASA website](https://oceancolor.gsfc.nasa.gov).

(Note that AIS is mandated only for distant-water fleets engaging on international voyages, and therefore, it does not cover small-scale/artisanal fisheries in coastal areas and within EEZ. Also, some areas have low AIS coverage, including Southeastern Asia and the Gulf of Mexico. AIS devices can be tampered with or turned off.)

(Note that high primary production does not always favor fish growth. A combination of high temperature and nutrient runoff can cause harmful algal blooms, leading to hypoxic dead zones and fish kills. If you are interested, you can explore the relationship between chlorophyll-a `chlor_a`, sea surface temperature `sst` and particulate organic carbon `poc`, in MODIS data.)